# Spark SQL Tutorial

### Difference between SparkContext , SparkSession and SqlContext

-  <b>SparkContext</b> : Used to be an entry point of spark application Prior to 2.0

-  <b>SparkSession</b> : has become an entry point to Spark to work with RDD, DataFrame, and Dataset, Since Spark 2.0  hence, PySpark applications start with initializing SparkSession 

- <b>SQLContext</b> :  is entry point of SparkSQL which can be received from sparkContext
 

In [1]:
# spark context 
from pyspark import SparkConf
from pyspark.context import SparkContext
# appName parameter is a name for your application to show on the cluster UI. 
#master is a Spark, Mesos or YARN cluster URL
conf = SparkConf().setAppName('appName').setMaster('local[*]')
#first thing a Spark program must do is to create a SparkContext object, which tells Spark how to access a cluster
sc = SparkContext(conf=conf)

In [2]:
# Spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
from pyspark import SparkConf,SQLContext
from pyspark.sql.functions import col
sqlContext = SQLContext(sc)

In [32]:
# Setting some environment variable  
SPARK_HOME = "file:/Users/kaustuv/spark-2.4.7/"


'file:/Users/kaustuv/spark-2.4.7/examples/src/main/resources/people.json'

## Spark SQL 

- Spark module for structured data processing, to execute SQL queries
- Spark SQL provides Spark, 
    - Structure information and 
    - Computation being performed and uses this to to perform extra optimizations


## DataFrame Creation

In [4]:
# read json to dataframe 
df = spark.read.json("file:/Users/kaustuv/spark-3.1.1/examples/src/main/resources/people.json")
# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# create a PySpark DataFrame from a list of rows
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2020, 1, 1), e=datetime(2020, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2020, 2, 1), e=datetime(2020, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2020, 3, 1), e=datetime(2020, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [6]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2020-01-01|2020-01-01 12:00:00|
|  2|3.0|string2|2020-02-01|2020-01-02 12:00:00|
|  4|5.0|string3|2020-03-01|2020-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [7]:
# Create a PySpark DataFrame with an explicit schema

df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df



DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [8]:
# Create a PySpark DataFrame from a pandas DataFrame

pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df



DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [9]:
# Create a PySpark DataFrame from an RDD consisting of a list of tuples.

rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
df



DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [10]:
# All DataFrames above result same.
df.show()
df.printSchema()


+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



## Viewing Data

In [11]:
# top rows of a DataFrame can be displayed using DataFrame.show()
df.show(1)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 1 row



In [12]:
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

Eager mode, meaning that all of the data is loaded into memory before the next step begins execution,
You can enable spark.sql.repl.eagerEval.enabled configuration for the eager evaluation of PySpark DataFrame in notebooks such as Jupyter. The number of rows to show can be controlled via spark.sql.repl.eagerEval.maxNumRows configuration.

In [13]:
# configuration for the eager evaluation 

spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [15]:
# shwing row verticaly, useful when rows are too long to show horizontally.
df.show(2, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
-RECORD 1------------------
 a   | 2                   
 b   | 3.0                 
 c   | string2             
 d   | 2000-02-01          
 e   | 2000-01-02 12:00:00 
only showing top 2 rows



In [16]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [17]:
# collects the distributed data to the driver side as the local data in Python
# throw an out-of-memory error when the dataset is too large to fit in the driver side
df.collect()

[Row(a=1, b=2.0, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3.0, c='string2', d=datetime.date(2000, 2, 1), e=datetime.datetime(2000, 1, 2, 12, 0)),
 Row(a=3, b=4.0, c='string3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 1, 3, 12, 0))]

In [18]:
# In order to avoid throwing an out-of-memory exception, use DataFrame.take() or DataFrame.tail().
df.take(1)

[Row(a=1, b=2.0, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0))]

### Converting Spark dataframe to panda dataframe

In [19]:
# DataFrame also provides the conversion back to a pandas DataFrame to leverage pandas APIs
pdr = df.toPandas()

In [20]:
pdr.describe()

,a,b
count,3.0,3.0
mean,2.0,3.0
std,1.0,1.0
min,1.0,2.0
25%,1.5,2.5
50%,2.0,3.0
75%,2.5,3.5
max,3.0,4.0


In [21]:
pdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   a       3 non-null      int64         
 1   b       3 non-null      float64       
 2   c       3 non-null      object        
 3   d       3 non-null      object        
 4   e       3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 248.0+ bytes


In [22]:
pdr.head()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


### Selecting and Accessing Data

In [23]:
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()


root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [25]:
# Select only the specified column
df.select("a").show()

+---+
|  a|
+---+
|  1|
|  2|
|  3|
+---+



In [26]:
# Select everybody, but increment the age by 1
df.select(df['a'], df['b'] + 1).show()

+---+-------+
|  a|(b + 1)|
+---+-------+
|  1|    3.0|
|  2|    4.0|
|  3|    5.0|
+---+-------+



In [27]:
# spark is lazily evaluated i.e. simply selecting a column does not trigger the computation
df.a

Column<b'a'>

In [28]:
type(df.c)

pyspark.sql.column.Column

In [29]:
type(df.c.isNull())

pyspark.sql.column.Column

In [30]:
# most of column-wise operations return Columns.
from pyspark.sql import Column
from pyspark.sql.functions import upper

type(df.c) == type(upper(df.c)) == type(df.c.isNull())

True

In [31]:
# DataFrame.select() takes the Column instances that returns another DataFrame.
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [32]:
# Assign new Column instance.
df.withColumn('upper_c', upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|upper_c|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|STRING2|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



In [33]:
# select a subset of rows using DataFrame.filter().

df.filter(df.a == 1).show()


+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



### Applying a Function

In [41]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [42]:
# Select column with condition
df.filter(df['a'] > 2).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



### Grouping Data

In [44]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()


+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [45]:
# Grouping and then applying the avg() function to the resulting groups.
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
|black|    6.0|   60.0|
| blue|    3.0|   30.0|
+-----+-------+-------+



In [46]:
df.groupby('color').sum().show()

+-----+-------+-------+
|color|sum(v1)|sum(v2)|
+-----+-------+-------+
|  red|     24|    240|
|black|      6|     60|
| blue|      6|     60|
+-----+-------+-------+



### Data Source : Getting Data in/out

In [59]:
# defining path
storage = "file://"
absolute_path= "/Users/kaustuv/Documents/Courses/Spark/pyspark_notebooks/"
relative_path = "data/"
json_file_name = "people.json"

file_path = storage+ absolute_path +relative_path
 

'file:///Users/kaustuv/Documents/Courses/Spark/pyspark_notebooks/data/people.json'

In [60]:

df = spark.read.load(storage+ absolute_path +relative_path +json_file_name, format="json")

#### Manually Specifying Options
- Can manually specify the data source that will be used along with any extra options t
- Data sources are specified by their fully qualified name : i.e. org.apache.spark.sql.parquet


In [61]:
# save as parqueat form
df.select("name", "age").write.save(storage+ absolute_path +relative_path + "out/namesAndAges.parquet", format="parquet")

#### CSV

In [94]:
csv_file_name = "people.csv"
df = spark.read.load(storage+ absolute_path +relative_path +csv_file_name ,
                     format="csv", sep=";", inferSchema="true", header="true")

In [65]:
# writing dataframe into CSV 

df.write.csv(file_path+'out/out_csv', header=True)

# reading  csv into data frame
csv_df = spark.read.csv(file_path+'out/out_csv', header=True)

In [66]:
csv_df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [67]:
csv_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)



#### JSON

In [72]:
# reading json into data frame 
json_df = spark.read.option("multiline","true").json(file_path+'data.json')

In [73]:
json_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- messageTime: long (nullable = true)
 |-- parameters: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- value: long (nullable = true)
 |-- roadNumber: long (nullable = true)



In [74]:
# to show all content
json_df.show(3, False)

+----------+-------------+-------------------------------------------------------------------------------+----------+
|customerId|messageTime  |parameters                                                                     |roadNumber|
+----------+-------------+-------------------------------------------------------------------------------+----------+
|ABC       |1597352330263|[[speed, 1597352329000, 0], [pcs, 1597352329000, 0], [notch, 1597352329000, 5]]|3802      |
+----------+-------------+-------------------------------------------------------------------------------+----------+



In [75]:
from pyspark.sql.functions import col
from pyspark.sql.functions import explode
json_df = json_df.withColumn("parameters", explode(json_df.parameters))
json_df.show(3, False)

+----------+-------------+-------------------------+----------+
|customerId|messageTime  |parameters               |roadNumber|
+----------+-------------+-------------------------+----------+
|ABC       |1597352330263|[speed, 1597352329000, 0]|3802      |
|ABC       |1597352330263|[pcs, 1597352329000, 0]  |3802      |
|ABC       |1597352330263|[notch, 1597352329000, 5]|3802      |
+----------+-------------+-------------------------+----------+



In [76]:
json_df.printSchema() 

root
 |-- customerId: string (nullable = true)
 |-- messageTime: long (nullable = true)
 |-- parameters: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |    |-- value: long (nullable = true)
 |-- roadNumber: long (nullable = true)



In [77]:
# 2 lvel sturcute dataframe into  single level structure
json_df = json_df.select("customerId","messageTime", 
                         col("parameters.name").alias("name"),
                         col("parameters.timestamp").alias("timestamp"),
                         col("parameters.value").alias("value"),"roadNumber")

json_df.printSchema()
json_df.show(20, False)


root
 |-- customerId: string (nullable = true)
 |-- messageTime: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- value: long (nullable = true)
 |-- roadNumber: long (nullable = true)

+----------+-------------+-----+-------------+-----+----------+
|customerId|messageTime  |name |timestamp    |value|roadNumber|
+----------+-------------+-----+-------------+-----+----------+
|ABC       |1597352330263|speed|1597352329000|0    |3802      |
|ABC       |1597352330263|pcs  |1597352329000|0    |3802      |
|ABC       |1597352330263|notch|1597352329000|5    |3802      |
+----------+-------------+-----+-------------+-----+----------+



In [78]:
# writing  df  as CSV 
# writing dataframe into CSV 
json_df.write.mode("overwrite").csv(file_path+'out_csv', header=True)



In [79]:
# writing  df as json 
json_df.write.json(file_path+'out/out_json')

#### Parquet

In [96]:
# write as parquet data 
df.write.parquet(file_path+'bar.parquet')

# read as parqueat
spark.read.parquet(file_path+'bar.parquet').show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



### ORC

The extra options are also used during write operation. For example, you can control bloom filters and dictionary encodings for ORC data sources. The following ORC example will create bloom filter and use dictionary encoding only for favorite_color.

<b> Bloom Filters : </b> Bloom filter is a probabilistic data structure designed to tell you, rapidly and memory-efficiently, whether an element is present in a set.It tells us that the element either definitely is not in the set or may be in the set.

<b> Dictionary Encodings : </b> A dictionary coder, also sometimes known as a substitution coder, is a class of lossless data compression algorithms which operate by searching for matches between the text to be compressed and a set of strings contained in a data structure (called the 'dictionary') maintained by the encoder.

In [85]:
df = spark.read.orc(file_path +"users.orc")
(df.write.format("orc")
    .option("orc.bloom.filter.columns", "favorite_color")
    .option("orc.dictionary.key.threshold", "1.0")
    .option("orc.column.encoding.direct", "name")
    .save(file_path+"out/users_with_options.orc"))

In [86]:
df.write.orc(file_path+'out/zoo.orc')
spark.read.orc(file_path+'out/zoo.orc').show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



### Avro 
- Since spark-avro module is external, there is no .avro API in DataFrameReader or DataFrameWriter.

- Different extension of Avro
    - .avsc - A JSON representation of an Avro schema (for a single object). This file is parsed by Avro libraries.
    - .avpr - A JSON representation of an Avro protocol (a collection of schemas)
    - .avdl - A code-like language that gets translated to .avsc or .avpr using the avro-tools.jar. This file is not used by Avro libraries, as far as I can tell.
    

### Parquet

In [103]:
# Run SQL on files directly
df = spark.sql("SELECT * FROM parquet.`file:///Users/kaustuv/spark-3.1.1/examples/src/main/resources/users.parquet`")

### Bucketing, Sorting and Partitioning

- For file-based data source, it is also possible to bucket and sort or partition the output. Bucketing and sorting are applicable only to persistent tables:

In [104]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [1]:
df.write.bucketBy(42, "name").sortBy("age").saveAsTable("people_bucketed").saveAsTable("people_bucketed")

In [2]:
 df.write.partitionBy("favorite_color").format("parquet").save("give_absolute_path")

it is possible to use both partitioning and bucketing for a single table:

In [ ]:
df = spark.read.parquet("file:///Users/kaustuv/spark-3.1.1/examples/src/main/resources/users.parquet")
(df
    .write
    .partitionBy("favorite_color")
    .bucketBy(42, "name")
    .saveAsTable("file:///Users/kaustuv/Documents/Courses/Spark/out/people_partitioned_bucketed"))

### Working with SQL: Running SQL Queries Programmatically

In [ ]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

In [ ]:
# register the DataFrame as a table and run a SQL easily as below:

In [ ]:
df.show()

In [ ]:
df.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
#  UDFs can be registered and invoked in SQL out of the box:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM tableA").show()


In [ ]:
#  SQL expressions can directly be mixed and used as PySpark columns.
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()

#### Global Temporary View
 - Temporary views in Spark SQL are session-scoped and 
 - will disappear if the session, that creates it, terminates

In [ ]:
# Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()

In [ ]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()

### Interoperating with RDDs


#### Inferring the Schema Using Reflection
- Spark SQL can convert an RDD of Row objects to a DataFrame, inferring the datatypes.


In [ ]:
from pyspark.sql import Row

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("file:///Users/kaustuv/spark-3.1.1/examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)
# Name: Justin

### Programmatically Specifying the Schema

- When a dictionary of kwargs cannot be defined ahead of time
- a DataFrame can be created programmatically with three steps.
    - Create an RDD of tuples or lists from the original RDD;
    - Create the schema represented by a StructType matching the structure of tuples or lists in the RDD created in the step 1.
    - Apply the schema to the RDD via createDataFrame method provided by SparkSession.


In [ ]:
# Import data types
from pyspark.sql.types import StringType, StructType, StructField

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("file:///Users/kaustuv/spark-3.1.1/examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
# Each line is converted to a tuple.
people = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "name age"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)

# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")

results.show()
 

## Performance Tuning

1. Caching Data In Memory :
    - Spark SQL can cache tables using an in-memory columnar format by calling spark.catalog.cacheTable("tableName") or dataFrame.cache()
    - It will automatically tune compression to minimize memory usage and GC pressure
    - To remove the table from memory call spark.catalog.uncacheTable("tableName") .
    - Configuration of in-memory caching can be done using the setConf method on SparkSession or by running SET key=value commands
    
    <br></br>
    
1. Using Configuration Options
    - spark.sql.files.maxPartitionBytes
    - spark.sql.files.maxPartitionBytes
    - spark.sql.files.minPartitionNum
    - spark.sql.broadcastTimeout
    - spark.sql.autoBroadcastJoinThreshold
    - spark.sql.shuffle.partitions
    - spark.sql.sources.parallelPartitionDiscovery.threshold
    - spark.sql.sources.parallelPartitionDiscovery.parallelism
    
    <br></br>
    
1. Join Strategy Hints for SQL Queries
    - BROADCAST, MERGE, SHUFFLE_HASH and SHUFFLE_REPLICATE_NL  are some of the joint strategy
    - Note that there is no guarantee that Spark will choose the join strategy specified in the hint since a specific strategy may not support all join types
    - sample usage : spark.table("src").join(spark.table("records").hint("broadcast"), "key").show() 
    
    <br></br>

1. Coalesce Hints for SQL Queries
    - Coalesce hints allows the Spark SQL users to control the number of output files hence can be used for performance tuning and reducing the number of output files.
    - The “COALESCE” hint only has a partition number as a parameter
    - The “REPARTITION” hint has a partition number, columns, or both of them as parameters.
    - The “REPARTITION_BY_RANGE” hint must have column names and a partition number is optional.
    - usage
        ```SELECT /*+ COALESCE(3) */ * FROM t
        SELECT /*+ REPARTITION(3) */ * FROM t
        SELECT /*+ REPARTITION(c) */ * FROM t
        SELECT /*+ REPARTITION(3, c) */ * FROM t
        SELECT /*+ REPARTITION_BY_RANGE(c) */ * FROM t
        SELECT /*+ REPARTITION_BY_RANGE(3, c) */ * FROM t
        ```
         <br></br>
       
1. Adaptive Query Execution
    - Adaptive Query Execution (AQE) is an optimization technique in Spark SQL that makes use of the runtime statistics to choose the most efficient query execution plan
    - Set spark.sql.adaptive.enabled to control  it on/off.
    - There are three major features in AQE, 
        
          1. Coalescing post-shuffle partitions
              Spark can pick the proper shuffle partition number at runtime once you set a large enough initial number of shuffle partitions via spark.sql.adaptive.coalescePartitions.initialPartitionNum configuration.
              
                   
          2. Converting sort-merge join to broadcast join 
                  AQE converts sort-merge join to broadcast hash join when the runtime statistics of any join side is smaller than the broadcast hash join threshold
                 
                  This is not as efficient as planning a broadcast hash join in the first place, but it’s better than keep doing the sort-merge join
          
          3. Skew join optimization.       
       
           This feature dynamically handles skew in sort-merge join by splitting (and replicating if needed) skewed tasks into roughly evenly sized tasks


## Data Types

### Supported Data Types


- Numeric types
     - ByteType: Represents 1-byte signed integer numbers. The range of numbers is from -128 to 127.
     - ShortType: Represents 2-byte signed integer numbers. The range of numbers is from -32768 to 32767.
     - IntegerType: Represents 4-byte signed integer numbers. The range of numbers is from -2147483648 to 2147483647.
     - LongType: Represents 8-byte signed integer numbers. The range of numbers is from - 9223372036854775808 to   9223372036854775807.
     
     - FloatType: Represents 4-byte single-precision floating point numbers.
     - DoubleType: Represents 8-byte double-precision floating point numbers.
     - DecimalType: Represents arbitrary-precision signed decimal numbers. Backed internally by java.math.BigDecimal. A BigDecimal consists of an arbitrary precision integer unscaled value and a 32-bit integer scale.
    
- String type
     - StringType: Represents character string values.
     - VarcharType(length): A variant of StringType which has a length limitation. Data writing will fail if the input string exceeds the length limitation. Note: this type can only be used in table schema, not functions/operators.
     - CharType(length): A variant of VarcharType(length) which is fixed length. Reading column of type CharType(n) always returns string values of length n. Char type column comparison will pad the short one to the longer length.

- Binary type
    - BinaryType: Represents byte sequence values.
    
- Boolean type
    - BooleanType: Represents boolean values.
    
- Datetime type
    - TimestampType: Represents values comprising values of fields year, month, day, hour, minute, and second, with the session local time-zone. The timestamp value represents an absolute point in time.
    - DateType: Represents values comprising values of fields year, month and day, without a time-zone.
    
    
- Complex types
    - ArrayType(elementType, containsNull): Represents values comprising a sequence of elements with the type of elementType. containsNull is used to indicate if elements in a ArrayType value can have null values.
    - MapType(keyType, valueType, valueContainsNull): Represents values comprising a set of key-value pairs. The data type of keys is described by keyType and the data type of values is described by valueType. For a MapType value, keys are not allowed to have null values. valueContainsNull is used to indicate if values of a MapType value can have null values.
        
    - StructType(fields): Represents values with the structure described by a sequence of StructFields (fields).
        - StructField(name, dataType, nullable): Represents a field in a StructType. The name of a field is indicated by name. The data type of a field is indicated by dataType. nullable is used to indicate if values of these fields can have null values.


In [ ]:
# All data types of Spark SQL are located in the package of pyspark.sql.types. You can access them by doing
from pyspark.sql.types import *

#### NaN Semantics

There is special handling for not-a-number (NaN) when dealing with float or double types that do not exactly match standard floating point semantics. Specifically:

-  NaN = NaN returns true.
- In aggregations, all NaN values are grouped together.
- NaN is treated as a normal value in join keys.
- NaN values go last when in ascending order, larger than any other numeric value.


In [ ]:
SELECT double('infinity') AS col;

In [ ]:
SELECT float('-inf') AS col;

### Spark -Sql



In [108]:
spark.sql("SELECT double('infinity') AS col").show()

+----+
| col|
+----+
|null|
+----+



In [118]:
spark.sql("SELECT float('-inf') AS col").show()

+----+
| col|
+----+
|null|
+----+



In [3]:
spark.sql("SELECT CAST('a' AS INT);").show()

In [4]:
spark.sql("SELECT float('NaN') AS col;").show()

In [ ]:
spark.sql("SELECT double('infinity') * 0 AS col;").show()

In [5]:
spark.sql("SELECT double('-infinity') * (-1234567) AS col;").show()

In [6]:
spark.sql("SELECT double('infinity') < double('NaN') AS col;").show()

In [7]:
spark.sql("SELECT double('NaN') = double('NaN') AS col;").show()

In [8]:
spark.sql("SELECT double('inf') = double('infinity') AS col;").show()

In [ ]:
spark.sql("CREATE TABLE test (c1 int, c2 double);")
          
          
          #INSERT INTO test VALUES (1, double('infinity'));INSERT INTO test VALUES (2, double('infinity'));INSERT INTO test VALUES (3, double('inf'));INSERT INTO test VALUES (4, double('-inf'));INSERT INTO test VALUES (5, double('NaN'));INSERT INTO test VALUES (6, double('NaN'));INSERT INTO test VALUES (7, double('-infinity'));")

### Difference between hive  using spark engine and  Spark-SQL

- Query  on hive  using spark engine uses hive libraries and its  function  where as  Spark-SQL uses  Spek-sql 
- Spark-sql can be run without  metastore also just by using dataframe
- jobs can be monotor in spark console where hive query are monito in application master console.


### SQL Functions

-  size: This function returns null for null input.
    
- element_at:
    - This function throws ArrayIndexOutOfBoundsException if using invalid indices.
    - This function throws NoSuchElementException if key does not exist in map.
- elt: This function throws ArrayIndexOutOfBoundsException if using invalid indices.
    
- parse_url: This function throws IllegalArgumentException if an input string is not a valid url.
    
- to_date: This function should fail with an exception if the input string can’t be parsed, or the pattern string is invalid.
    
- to_timestamp: This function should fail with an exception if the input string can’t be parsed, or the pattern string is invalid.
    
- unix_timestamp: This function should fail with an exception if the input string can’t be parsed, or the pattern string is invalid.
    
- to_unix_timestamp: This function should fail with an exception if the input string can’t be parsed, or the pattern string is invalid.
    
- make_date: This function should fail with an exception if the result date is invalid.
    
- make_timestamp: This function should fail with an exception if the result timestamp is invalid.
    
- make_interval: This function should fail with an exception if the result interval is invalid.


### SQL Operators


- array_col[index]: This operator throws ArrayIndexOutOfBoundsException if using invalid indices.
    
- map_col[key]: This operator throws NoSuchElementException if key does not exist in map.
    
- CAST(string_col AS TIMESTAMP): This operator should fail with an exception if the input string can’t be parsed.


### SQL Keywords

- Reserved keywords: Keywords that are reserved and can’t be used as identifiers for table, view, column, function, alias, etc.
- Non-reserved keywords: Keywords that have a special meaning only in particular contexts and can be used as identifiers in other contexts. For example, EXPLAIN SELECT ... is a command, but EXPLAIN can be used as identifiers in other places.

### Built-in Functions

#### Aggregate Functions

Function 	Description

- any(expr): 	Returns true if at least one value of `expr` is true.

- approx_count_distinct(expr[, relativeSD]): 	Returns the estimated cardinality by HyperLogLog++. `relativeSD` defines the maximum relative standard deviation allowed.

- approx_percentile(col, percentage [, accuracy]) :	Returns the approximate `percentile` of the numeric column `col` which is the smallest value in the ordered `col` values (sorted from least to greatest) such that no more than `percentage` of `col` values is less than the value or equal to that value. The value of percentage must be between 0.0 and 1.0. The `accuracy` parameter (default: 10000) is a positive numeric literal which controls approximation accuracy at the cost of memory. Higher value of `accuracy` yields better accuracy, `1.0/accuracy` is the relative error of the approximation. When `percentage` is an array, each value of the percentage array must be between 0.0 and 1.0. In this case, returns the approximate percentile array of column `col` at the given percentage array.

- avg(expr): 	Returns the mean calculated from values of a group.

- bit_and(expr): 	Returns the bitwise AND of all non-null input values, or null if none.
- bit_or(expr): 	Returns the bitwise OR of all non-null input values, or null if none.
- bit_xor(expr): 	Returns the bitwise XOR of all non-null input values, or null if none.
- bool_and(expr): 	Returns true if all values of `expr` are true.
- bool_or(expr): 	Returns true if at least one value of `expr` is true.
- collect_list(expr): 	Collects and returns a list of non-unique elements.
- collect_set(expr): 	Collects and returns a set of unique elements.
- corr(expr1, expr2): 	Returns Pearson coefficient of correlation between a set of number pairs.
- count(*): 	Returns the total number of retrieved rows, including rows containing null.
- count(expr[, expr...]):	Returns the number of rows for which the supplied expression(s) are all non-null.
- count(DISTINCT expr[, expr...]): 	Returns the number of rows for which the supplied expression(s) are unique and non-null.
- count_if(expr): 	Returns the number of `TRUE` values for the expression.
- count_min_sketch(col, eps, confidence, seed): 	Returns a count-min sketch of a column with the given esp, confidence and seed. The result is an array of bytes, which can be deserialized to a `CountMinSketch` before usage. Count-min sketch is a probabilistic data structure used for cardinality estimation using sub-linear space.

- covar_pop(expr1, expr2): 	Returns the population covariance of a set of number pairs.

- covar_samp(expr1, expr2): 	Returns the sample covariance of a set of number pairs.

- every(expr): 	Returns true if all values of `expr` are true.

- first(expr[, isIgnoreNull]): 	Returns the first value of `expr` for a group of rows. If `isIgnoreNull` is true, returns only non-null values.
- first_value(expr[, isIgnoreNull]) :	Returns the first value of `expr` for a group of rows. If `isIgnoreNull` is true, returns only non-null values.

- kurtosis(expr): 	Returns the kurtosis value calculated from values of a group.

- last(expr[, isIgnoreNull]): 	Returns the last value of `expr` for a group of rows. If `isIgnoreNull` is true, returns only non-null values

- last_value(expr[, isIgnoreNull]): 	Returns the last value of `expr` for a group of rows. If `isIgnoreNull` is true, returns only non-null values

- max(expr) :	Returns the maximum value of `expr`.
- max_by(x, y): 	Returns the value of `x` associated with the maximum value of `y`.
- mean(expr): 	Returns the mean calculated from values of a group.
- min(expr): 	Returns the minimum value of `expr`.
- min_by(x, y): 	Returns the value of `x` associated with the minimum value of `y`.
- percentile(col, percentage [, frequency]): 	Returns the exact percentile value of numeric column `col` at the given percentage. The value of percentage must be between 0.0 and 1.0. The value of frequency should be positive integral
- percentile(col, array(percentage1 [, percentage2]...) [, frequency]): 	Returns the exact percentile value array of numeric column `col` at the given percentage(s). Each value of the percentage array must be between 0.0 and 1.0. The value of frequency should be positive integral
- percentile_approx(col, percentage [, accuracy]): 	Returns the approximate `percentile` of the numeric column `col` which is the smallest value in the ordered `col` values (sorted from least to greatest) such that no more than `percentage` of `col` values is less than the value or equal to that value. The value of percentage must be between 0.0 and 1.0. The `accuracy` parameter (default: 10000) is a positive numeric literal which controls approximation accuracy at the cost of memory. Higher value of `accuracy` yields better accuracy, `1.0/accuracy` is the relative error of the approximation. When `percentage` is an array, each value of the percentage array must be between 0.0 and 1.0. In this case, returns the approximate percentile array of column `col` at the given percentage array.
- skewness(expr) :	Returns the skewness value calculated from values of a group.
- some(expr): 	Returns true if at least one value of `expr` is true.
- std(expr): 	Returns the sample standard deviation calculated from values of a group.
- stddev(expr): 	Returns the sample standard deviation calculated from values of a group.
- stddev_pop(expr): 	Returns the population standard deviation calculated from values of a group.
- stddev_samp(expr): 	Returns the sample standard deviation calculated from values of a group.
- sum(expr): 	Returns the sum calculated from values of a group.
- var_pop(expr): 	Returns the population variance calculated from values of a group.
- var_samp(expr): 	Returns the sample variance calculated from values of a group.
- variance(expr): 	Returns the sample variance calculated from values of a group.

#### Window Functions


 
- cume_dist(): 	Computes the position of a value relative to all values in the partition.

- dense_rank(): 	Computes the rank of a value in a group of values. The result is one plus the previously assigned rank value. Unlike the function rank, dense_rank will not produce gaps in the ranking sequence.

- lag(input[, offset[, default]]): 	Returns the value of `input` at the `offset`th row before the current row in the window. The default value of `offset` is 1 and the default value of `default` is null. If the value of `input` at the `offset`th row is null, null is returned. If there is no such offset row (e.g., when the offset is 1, the first row of the window does not have any previous row), `default` is returned.

- lead(input[, offset[, default]]): 	Returns the value of `input` at the `offset`th row after the current row in the window. The default value of `offset` is 1 and the default value of `default` is null. If the value of `input` at the `offset`th row is null, null is returned. If there is no such an offset row (e.g., when the offset is 1, the last row of the window does not have any subsequent row), `default` is returned.

- nth_value(input[, offset]): 	Returns the value of `input` at the row that is the `offset`th row from beginning of the window frame. Offset starts at 1. If ignoreNulls=true, we will skip nulls when finding the `offset`th row. Otherwise, every row counts for the `offset`. If there is no such an `offset`th row (e.g., when the offset is 10, size of the window frame is less than 10), null is returned.

- ntile(n): 	Divides the rows for each window partition into `n` buckets ranging from 1 to at most `n`.

- percent_rank(): 	Computes the percentage ranking of a value in a group of values.

- rank() :	Computes the rank of a value in a group of values. The result is one plus the number of rows preceding or equal to the current row in the ordering of the partition. The values will produce gaps in the sequence.

- row_number(): 	Assigns a unique, sequential number to each row, starting with one, according to the ordering of rows within the window partition.


#### Array Functions


- array_contains(array, value): 	Returns true if the array contains the value.
- array_distinct(array):	Removes duplicate values from the array.
- array_except(array1, array2): 	Returns an array of the elements in array1 but not in array2, without duplicates.
- array_intersect(array1, array2) :	Returns an array of the elements in the intersection of array1 and array2, without duplicates.
- array_join(array, delimiter[, nullReplacement]): 	Concatenates the elements of the given array using the delimiter and an optional string to replace nulls. If no value is set for nullReplacement, any null value is filtered.
- array_max(array): 	Returns the maximum value in the array. NULL elements are skipped.
- array_min(array): 	Returns the minimum value in the array. NULL elements are skipped.
- array_position(array, element): 	Returns the (1-based) index of the first element of the array as long.
- array_remove(array, element): 	Remove all elements that equal to element from array.
- array_repeat(element, count): 	Returns the array containing element count times.
- array_union(array1, array2): 	Returns an array of the elements in the union of array1 and array2, without duplicates.

- arrays_overlap(a1, a2):	Returns true if a1 contains at least a non-null element present also in a2. If the arrays have no common element and they are both non-empty and either of them contains a null element null is returned, false otherwise.

- arrays_zip(a1, a2, ...): 	Returns a merged array of structs in which the N-th struct contains all N-th values of input arrays.

- concat(col1, col2, ..., colN): 	Returns the concatenation of col1, col2, ..., colN.

- flatten(arrayOfArrays):	Transforms an array of arrays into a single array.

- reverse(array): 	Returns a reversed string or an array with reverse order of elements.

- sequence(start, stop, step): 	Generates an array of elements from start to stop (inclusive), incrementing by step. The type of the returned elements is the same as the type of argument expressions. Supported types are: byte, short, integer, long, date, timestamp. The start and stop expressions must resolve to the same type. If start and stop expressions resolve to the 'date' or 'timestamp' type then the step expression must resolve to the 'interval' type, otherwise to the same type as the start and stop expressions.

- shuffle(array): 	Returns a random permutation of the given array.

- slice(x, start, length): 	Subsets array x starting from index start (array indices start at 1, or starting from the end if start is negative) with the specified length.

- sort_array(array[, ascendingOrder]): 	Sorts the input array in ascending or descending order according to the natural ordering of the array elements. Null elements will be placed at the beginning of the returned array in ascending order or at the end of the returned array in descending order.

#### Map Functions

- map_concat(map, ...): 	Returns the union of all the given maps

- map_entries(map): 	Returns an unordered array of all entries in the given map.

- map_from_entries(arrayOfEntries): 	Returns a map created from the given array of entries.

- map_keys(map): 	Returns an unordered array containing the keys of the map.

- map_values(map): 	Returns an unordered array containing the values of the map.

### Date and Timestamp Functions

- add_months(start_date, num_months): 	Returns the date that is `num_months` after `start_date`.

- current_date(): 	Returns the current date at the start of query evaluation. All calls of current_date within the same query return the same value.

- current_date: 	Returns the current date at the start of query evaluation.

- current_timestamp(): 	Returns the current timestamp at the start of query evaluation. All calls of current_timestamp within the same query return the same value.

- current_timestamp :	Returns the current timestamp at the start of query evaluation.

- current_timezone(): 	Returns the current session local timezone.

- date_add(start_date, num_days): 	Returns the date that is `num_days` after `start_date`.

- date_format(timestamp, fmt): 	Converts `timestamp` to a value of string in the format specified by the date format `fmt`.

- date_from_unix_date(days): 	Create date from the number of days since 1970-01-01.

- date_part(field, source): 	Extracts a part of the date/timestamp or interval source.

- date_sub(start_date, num_days): 	Returns the date that is `num_days` before `start_date`.

- date_trunc(fmt, ts): 	Returns timestamp `ts` truncated to the unit specified by the format model `fmt`.

- datediff(endDate, startDate): 	Returns the number of days from `startDate` to `endDate`.

- dayofweek(date): 	Returns the day of the week for date/timestamp (1 = Sunday, 2 = Monday, ..., 7 = Saturday).

- dayofyear(date): 	Returns the day of year of the date/timestamp.

- from_unixtime(unix_time[, fmt]): 	Returns `unix_time` in the specified `fmt`.

- from_utc_timestamp(timestamp, timezone): 	Given a timestamp like '2017-07-14 02:40:00.0', interprets it as a time in UTC, and renders that time as a timestamp in the given time zone. For example, 'GMT+1' would yield '2017-07-14 03:40:00.0'.

- hour(timestamp): 	Returns the hour component of the string/timestamp.

- last_day(date): 	Returns the last day of the month which the date belongs to.

- make_date(year, month, day): 	Create date from year, month and day fields.

- make_timestamp(year, month, day, hour, min, sec[, timezone]): 	Create timestamp from year, month, day, hour, min, sec and timezone fields.

- minute(timestamp): 	Returns the minute component of the string/timestamp.

- month(date): 	Returns the month component of the date/timestamp.

- months_between(timestamp1, timestamp2[, roundOff]): 	If `timestamp1` is later than `timestamp2`, then the result is positive. If `timestamp1` and `timestamp2` are on the same day of month, or both are the last day of month, time of day will be ignored. Otherwise, the difference is calculated based on 31 days per month, and rounded to 8 digits unless roundOff=false.

- next_day(start_date, day_of_week): 	Returns the first date which is later than `start_date` and named as indicated.

- now(): 	Returns the current timestamp at the start of query evaluation.

- quarter(date): 	Returns the quarter of the year for date, in the range 1 to 4.

- second(timestamp): 	Returns the second component of the string/timestamp.

- timestamp_micros(microseconds):	Creates timestamp from the number of microseconds since UTC epoch.

- timestamp_millis(milliseconds): 	Creates timestamp from the number of milliseconds since UTC epoch.

- timestamp_seconds(seconds) :	Creates timestamp from the number of seconds (can be fractional) since UTC epoch.

- to_date(date_str[, fmt]): 	Parses the `date_str` expression with the `fmt` expression to a date. Returns null with invalid input. By default, it follows casting rules to a date if the `fmt` is omitted.

- to_timestamp(timestamp_str[, fmt]) :	Parses the `timestamp_str` expression with the `fmt` expression to a timestamp. Returns null with invalid input. By default, it follows casting rules to a timestamp if the `fmt` is omitted.

- to_unix_timestamp(timeExp[, fmt]):	Returns the UNIX timestamp of the given time.

- to_utc_timestamp(timestamp, timezone): 	Given a timestamp like '2017-07-14 02:40:00.0', interprets it as a time in the given time zone, and renders that time as a timestamp in UTC. For example, 'GMT+1' would yield '2017-07-14 01:40:00.0'.

- trunc(date, fmt): 	Returns `date` with the time portion of the day truncated to the unit specified by the format model `fmt`.

- unix_date(date): 	Returns the number of days since 1970-01-01.

- unix_micros(timestamp): 	Returns the number of microseconds since 1970-01-01 00:00:00 UTC.

- unix_millis(timestamp): 	Returns the number of milliseconds since 1970-01-01 00:00:00 UTC. Truncates higher levels of precision.

- unix_seconds(timestamp): 	Returns the number of seconds since 1970-01-01 00:00:00 UTC. Truncates higher levels of precision.

- unix_timestamp([timeExp[, fmt]]): 	Returns the UNIX timestamp of current or specified time.

- weekday(date): 	Returns the day of the week for date/timestamp (0 = Monday, 1 = Tuesday, ..., 6 = Sunday).

- weekofyear(date): 	Returns the week of the year of the given date. A week is considered to start on a Monday and week 1 is the first week with >3 days.

- year(date): 	Returns the year component of the date/timestamp.

#### JSON Functions

- from_json(jsonStr, schema[, options]): 	Returns a struct value with the given `jsonStr` and `schema`.
- get_json_object(json_txt, path): 	Extracts a json object from `path`.
- json_array_length(jsonArray): 	Returns the number of elements in the outmost JSON array.
- json_object_keys(json_object): 	Returns all the keys of the outmost JSON object as an array.
- json_tuple(jsonStr, p1, p2, ..., pn): 	Returns a tuple like the function get_json_object, but it takes multiple names. All the input parameters and output column types are string.

- schema_of_json(json[, options]): 	Returns schema in the DDL format of JSON string.
- to_json(expr[, options]): 	Returns a JSON string with a given struct value

### Scalar User Defined Functions (UDFs)

Scalar User-Defined Functions (UDFs) are user-programmable routines that act on one row. This documentation lists the classes that are required for creating and registering UDFs. It also contains examples that demonstrate how to define and register UDFs and invoke them in Spark SQL.

Available only in java & Scala

#### User Defined Aggregate Functions (UDAFs)

User-Defined Aggregate Functions (UDAFs) are user-programmable routines that act on multiple rows at once and return a single aggregated value as a result. This documentation lists the classes that are required for creating and registering UDAFs. It also contains examples that demonstrate how to define and register UDAFs in Scala and invoke them in Spark SQL.m
Available only in java & Scala

Identifiers : An identifier is a string used to identify a database object such as a table, view, schema, column, etc. Spark SQL has regular identifiers and delimited identifiers, which are enclosed within backticks. Both regular identifiers and delimited identifiers are case-insensitive


Literals : A literal (also known as a constant) represents a fixed data value. Spark SQL supports the following literals

### Data Definition Statements
- ALTER DATABASE
- ALTER TABLE
- ALTER VIEW
- CREATE DATABASE
- CREATE FUNCTION
- CREATE TABLE
- CREATE VIEW
- DROP DATABASE
- DROP FUNCTION
- DROP TABLE
- DROP VIEW
- TRUNCATE TABLE
- REPAIR TABLE
- USE DATABASE


### Data Manipulation Statements

- INSERT 
    - INSERT INTO statement
    - INSERT OVERWRITE statement
    - INSERT OVERWRITE DIRECTORY statement
    - INSERT OVERWRITE DIRECTORY with Hive format statement
    
- LOAD
    - LOAD DATA statement loads the data into a Hive serde table from the user specified directory or file. If a directory is specified then all the files from the directory are loaded. If a file is specified then only the single file is loaded. Additionally


### Data Retrieval


- SELECT Statement : retrieve rows from one or more tables according to the specified clauses
     - WHERE Clause
     - GROUP BY Clause
     - HAVING Clause
     - ORDER BY Clause
     - SORT BY Clause
     - CLUSTER BY Clause
     - DISTRIBUTE BY Clause
     - LIMIT Clause
     - Common Table Expression
     - Hints
     - Inline Table
     - File
     - JOIN
     - LIKE Predicate
     - Set Operators
     - TABLESAMPLE
     - Table-valued Function
     - Window Function
     - CASE Clause
     - PIVOT Clause
     - LATERAL VIEW Clause
     - TRANSFORM Clause
     
- EXPLAIN Statement : generate logical and physical plan for a given query


### Auxiliary Statements

- ANALYZE : The ANALYZE TABLE statement collects statistics about the table to be used by the query optimizer to find a better query execution plan.

- CACHE :
    - CACHE TABLE
    - UNCACHE TABLE
    - CLEAR CACHE
    - REFRESH TABLE
    - REFRESH FUNCTION
    - REFRESH

- DESCRIBE :
    - DESCRIBE DATABASE
    - DESCRIBE TABLE
    - DESCRIBE FUNCTION
    - DESCRIBE QUERY

- SHOW : 
   -  SHOW COLUMNS
   -  SHOW CREATE TABLE
   -  SHOW DATABASES
   -  SHOW FUNCTIONS
   -  SHOW PARTITIONS
   -  SHOW TABLE EXTENDED
   -  SHOW TABLES
   -  SHOW TBLPROPERTIES
   -  SHOW VIEWS


- CONFIGURATION MANAGEMENT 
    - SET
    - RESET
    - SET TIME ZONE

- RESOURCE MANAGEMENT
    - ADD FILE
    - ADD JAR
    - LIST FILE
    - LIST JAR

